### 1. Getting Data

In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns',None) # exibir todas as colunas do meu df
pd.set_option("display.max_seq_items", None) # exibir todas as linhas no meu for

In [155]:
#data = 'https://raw.githubusercontent.com/bruno-lima98/ml-zoomcamp-homework/refs/heads/main/01-classes/03-classification/telco_data.csv'
#!wget $data -O telco_data.csv

In [156]:
df = pd.read_csv('telco_data.csv')

### 2. Data Prep

In [157]:
# Padronizamos a escrita das colunas, títulos e informações

df.columns = df.columns.str.lower().str.replace(' ','_')
categorical_colums = list(df.dtypes[df.dtypes == 'object'].index)

for i in categorical_colums:
    df[i] = df[i].str.lower().str.replace(' ','_')

In [158]:
# Ajustamos a coluna totalcharges que é número mas estava como string por possuir espaços vazios

df['totalcharges'] = pd.to_numeric(df['totalcharges'].str.replace('_','0'))

In [159]:
# Ajustamos nossa variável target binária para 0/1 no lugar yes/no

df['churn'] = (df['churn'] == 'yes').astype('int')

In [160]:
df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-vhveg,female,0,yes,no,1,no,no_phone_service,dsl,no,yes,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,0
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,no,yes,no,no,no,one_year,no,mailed_check,56.95,1889.50,0
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,yes,no,no,no,no,month-to-month,yes,mailed_check,53.85,108.15,1
3,7795-cfocw,male,0,no,no,45,no,no_phone_service,dsl,yes,no,yes,yes,no,no,one_year,no,bank_transfer_(automatic),42.30,1840.75,0
4,9237-hqitu,female,0,no,no,2,yes,no,fiber_optic,no,no,no,no,no,no,month-to-month,yes,electronic_check,70.70,151.65,1


### 3. Setting up Validation Framework

In [161]:
from sklearn.model_selection import train_test_split
# Separamos o nosso dataset em Treino, Validação e Teste usando uma biblioteca diretamente

In [162]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [163]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [164]:
len(df_full_train), len(df_train), len(df_test), len(df_val)

(5634, 4225, 1409, 1409)

In [165]:
# Resetamos os indíces dos DFs apenas para não ficar embaralhado

df_full_train = df_full_train.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [166]:
y_full_train = df_full_train['churn'].values
y_train = df_train['churn'].values
y_val = df_val['churn'].values
y_test = df_test['churn'].values

In [167]:
# del df_full_train['churn']
del df_train['churn']
del df_val['churn']
del df_test['churn']

### 4. EDA

In [168]:
df_full_train.isnull().sum()
# Verificamos que não temos mais valores nulos no df

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [169]:
global_churn_rate = df_full_train['churn'].value_counts(normalize=True)[1]
# Essa é uma forma de filtrar os valores. Calculei o % de cada categoria e depois chamei o valor da categoria que vale 1.

round(global_churn_rate,2)

np.float64(0.27)

In [170]:

numerical = ['tenure','monthlycharges','totalcharges']
categorical = [
                'gender', 'seniorcitizen', 'partner', 'dependents',
                'phoneservice', 'multiplelines', 'internetservice',
                'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
                'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
                'paymentmethod']

In [171]:
for i in categorical:
    print(f'{i:<17} | Total de categorias: {df_full_train[i].nunique()} | Categorias: {list(df_full_train[i].unique())}')

gender            | Total de categorias: 2 | Categorias: ['male', 'female']
seniorcitizen     | Total de categorias: 2 | Categorias: [np.int64(0), np.int64(1)]
partner           | Total de categorias: 2 | Categorias: ['yes', 'no']
dependents        | Total de categorias: 2 | Categorias: ['yes', 'no']
phoneservice      | Total de categorias: 2 | Categorias: ['yes', 'no']
multiplelines     | Total de categorias: 3 | Categorias: ['no', 'yes', 'no_phone_service']
internetservice   | Total de categorias: 3 | Categorias: ['no', 'dsl', 'fiber_optic']
onlinesecurity    | Total de categorias: 3 | Categorias: ['no_internet_service', 'yes', 'no']
onlinebackup      | Total de categorias: 3 | Categorias: ['no_internet_service', 'yes', 'no']
deviceprotection  | Total de categorias: 3 | Categorias: ['no_internet_service', 'yes', 'no']
techsupport       | Total de categorias: 3 | Categorias: ['no_internet_service', 'yes', 'no']
streamingtv       | Total de categorias: 3 | Categorias: ['no_internet_ser

### 5. Featuring Engineering

In [172]:
male_churn_rate = df_full_train[df_full_train['gender']=='male']['churn'].mean()
female_churn_rate = df_full_train[df_full_train['gender']=='female']['churn'].mean()

print(f'Taxa de Churn Global = {global_churn_rate:.4f} | Taxa (homens) = {male_churn_rate:.4f} | Taxa (mulheres) = {female_churn_rate:.4f}')

# Podemos ver que as taxas entre homens e mulheres são bem parecidas, indicando que talvez a variável de sexo não importe tanto

Taxa de Churn Global = 0.2700 | Taxa (homens) = 0.2632 | Taxa (mulheres) = 0.2768


In [173]:
partner_churn = df_full_train[df_full_train['partner']=='yes']['churn'].mean()
no_partner_churn = df_full_train[df_full_train['partner']=='no']['churn'].mean()

print(f'Taxa de Churn Global = {global_churn_rate:.4f} | Taxa (Com Parceiro) = {partner_churn:.4f} | Taxa (Sem Parceiro) = {no_partner_churn:.4f}')

# Podemos ver que há uma diferença considerável entre o churn em pessoas com parceiro e sem parceiro, indicando que essa talvez possa ser uma variável importante

Taxa de Churn Global = 0.2700 | Taxa (Com Parceiro) = 0.2050 | Taxa (Sem Parceiro) = 0.3298


In [174]:
a = ((df_full_train.groupby('paymentmethod')['churn'].mean() / global_churn_rate - 1).round(4)*100)
a[1]

/tmp/ipykernel_2927/3938576978.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  a[1]


np.float64(-39.129999999999995)

In [175]:
for i in categorical:
    a = (df_full_train.groupby(i)['churn'].mean() / global_churn_rate - 1).round(4)*100
    print(f'{i}')
    for j in range(len(a)):
        print(f'- {a.index[j]}: {a.values[j].round(2)}')

gender
- female: 2.54
- male: -2.5
seniorcitizen
- 0: -10.26
- 1: 53.12
partner
- no: 22.17
- yes: -24.05
dependents
- no: 16.22
- yes: -38.63
phoneservice
- no: -10.61
- yes: 1.14
multiplelines
- no: -4.65
- no_phone_service: -10.61
- yes: 7.69
internetservice
- dsl: -28.75
- fiber_optic: 57.49
- no: -71.18
onlinesecurity
- no: 55.92
- no_internet_service: -71.18
- yes: -43.24
onlinebackup
- no: 49.77
- no_internet_service: -71.18
- yes: -19.53
deviceprotection
- no: 46.64
- no_internet_service: -71.18
- yes: -14.65
techsupport
- no: 55.17
- no_internet_service: -71.18
- yes: -40.76
streamingtv
- no: 26.99
- no_internet_service: -71.18
- yes: 12.13
streamingmovies
- no: 25.54
- no_internet_service: -71.18
- yes: 13.82
contract
- month-to-month: 59.91
- one_year: -55.34
- two_year: -89.53
paperlessbilling
- no: -36.26
- yes: 25.26
paymentmethod
- bank_transfer_(automatic): -37.71
- credit_card_(automatic): -39.13
- electronic_check: 68.87
- mailed_check: -28.19


In [176]:
for i in categorical:
    df_group = df_full_train.groupby(i)['churn'].agg(['count','mean'])
    df_group['mean'] = (df_group['mean']).round(4)
    df_group['diff_abs'] = ((df_group['mean'] - (global_churn_rate))).round(4)
    df_group['risk'] = ((df_group['mean'] / (global_churn_rate))).round(4)

In [177]:
from IPython.display import display

for i in categorical:
    print(i)
    df_group_disp = df_full_train.groupby(i)['churn'].agg(['count','mean'])
    df_group_disp['mean'] = (df_group_disp['mean']*100).round(2)
    df_group_disp['diff_abs (%)'] = ((df_group_disp['mean'] - (global_churn_rate*100))).round(2)
    df_group_disp['risk (%)'] = ((df_group_disp['mean'] / (global_churn_rate*100) - 1)*100).round(2)

    df_group_disp = df_group_disp.rename(columns={'mean': 'mean (%)'})
    
    df_group_disp = df_group_disp.reset_index()

    display(df_group_disp)
    print()

gender


,gender,count,mean (%),diff_abs (%),risk (%)
0,female,2796,27.68,0.68,2.53
1,male,2838,26.32,-0.68,-2.51



seniorcitizen


,seniorcitizen,count,mean (%),diff_abs (%),risk (%)
0,0,4722,24.23,-2.77,-10.25
1,1,912,41.34,14.34,53.13



partner


...

,contract,count,mean (%),diff_abs (%),risk (%)
0,month-to-month,3104,43.17,16.17,59.91
1,one_year,1186,12.06,-14.94,-55.33
2,two_year,1344,2.83,-24.17,-89.52



paperlessbilling


,paperlessbilling,count,mean (%),diff_abs (%),risk (%)
0,no,2313,17.21,-9.79,-36.25
1,yes,3321,33.82,6.82,25.27



paymentmethod


,paymentmethod,count,mean (%),diff_abs (%),risk (%)
0,bank_transfer_(automatic),1219,16.82,-10.18,-37.70
1,credit_card_(automatic),1217,16.43,-10.57,-39.14
2,electronic_check,1893,45.59,18.59,68.87
3,mailed_check,1305,19.39,-7.61,-28.18


### 6. Feature Importance: Mutual Information

In [178]:
from sklearn.metrics import mutual_info_score

In [179]:
mutual_info_score(df_full_train['churn'],df_full_train['contract'])
# Essa função no permite ver a importância de uma variável em relação à outra.
# Dessa forma, se olhamos uma variável e comparamos com o churn, quanto maior o número, mais importância tem aquela variável sobre o churn

0.0983203874041556

In [180]:
mutual = {}
for i in categorical:
    a = round(mutual_info_score(df_full_train['churn'],df_full_train[i])*100,2)
    mutual[i] = a
df_mutual = pd.DataFrame(list(mutual.items()), columns=['feature', 'mutual_info (%)'])
df_mutual = df_mutual.sort_values(by="mutual_info (%)", ascending=False).reset_index(drop=True)
df_mutual

,feature,mutual_info (%)
0,contract,9.83
1,onlinesecurity,6.31
2,techsupport,6.10
3,internetservice,5.59
4,onlinebackup,4.69
5,deviceprotection,4.35
6,paymentmethod,4.32
7,streamingtv,3.19
8,streamingmovies,3.16
9,paperlessbilling,1.76


### 7. Correlation

In [181]:
df_full_train[numerical].corrwith(df_full_train['churn'])

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

### 8. One-Hot Encoding

In [182]:
from sklearn.feature_extraction import DictVectorizer

In [183]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

In [184]:
dv = DictVectorizer(sparse=False)

In [185]:
dv.fit(train_dicts)
list(dv.get_feature_names_out())

['contract=month-to-month',
 'contract=one_year',
 'contract=two_year',
 'dependents=no',
 'dependents=yes',
 'deviceprotection=no',
 'deviceprotection=no_internet_service',
 'deviceprotection=yes',
 'gender=female',
 'gender=male',
 'internetservice=dsl',
 'internetservice=fiber_optic',
 'internetservice=no',
 'monthlycharges',
 'multiplelines=no',
 'multiplelines=no_phone_service',
 'multiplelines=yes',
 'onlinebackup=no',
 'onlinebackup=no_internet_service',
 'onlinebackup=yes',
 'onlinesecurity=no',
 'onlinesecurity=no_internet_service',
 'onlinesecurity=yes',
 'paperlessbilling=no',
 'paperlessbilling=yes',
 'partner=no',
 'partner=yes',
 'paymentmethod=bank_transfer_(automatic)',
 'paymentmethod=credit_card_(automatic)',
 'paymentmethod=electronic_check',
 'paymentmethod=mailed_check',
 'phoneservice=no',
 'phoneservice=yes',
 'seniorcitizen',
 'streamingmovies=no',
 'streamingmovies=no_internet_service',
 'streamingmovies=yes',
 'streamingtv=no',
 'streamingtv=no_internet_servic

In [186]:
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

In [187]:
X_train.shape, X_val.shape

((4225, 45), (1409, 45))

### 9. Logistic Regression

In [188]:
from sklearn.linear_model import LogisticRegression

In [189]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [190]:
model.intercept_[0]

np.float64(-0.12066865593034165)

In [191]:
model.coef_[0].round(3)

array([ 0.639, -0.016, -0.734, -0.018, -0.093,  0.061, -0.11 , -0.062,
       -0.036, -0.075, -0.365,  0.364, -0.11 ,  0.   , -0.243,  0.118,
        0.014,  0.062, -0.11 , -0.063,  0.231, -0.11 , -0.232, -0.238,
        0.127, -0.123,  0.012, -0.087, -0.037,  0.066, -0.053,  0.118,
       -0.229,  0.193, -0.102, -0.11 ,  0.101, -0.062, -0.11 ,  0.06 ,
        0.181, -0.11 , -0.182, -0.068,  0.   ])

In [192]:
y_pred = model.predict_proba(X_val)[:,1]

In [193]:
churn_decision = (y_pred >= 0.5)

In [196]:
(y_val == churn_decision.astype('int')).mean()

np.float64(0.8055358410220014)